In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls "/content/drive/My Drive/Colab Notebooks"

'Copy of mnist.ipynb'	     Untitled12.ipynb   Untitled24.ipynb
'Copy of QuantumCNN.ipynb'   Untitled13.ipynb   Untitled25.ipynb
 FTP-Patator.csv	     Untitled14.ipynb   Untitled26.ipynb
 olivetti_faces.npy	     Untitled15.ipynb   Untitled2.ipynb
 olivetti_faces_target.npy   Untitled16.ipynb   Untitled3.ipynb
 ORL_faces.npz		     Untitled17.ipynb   Untitled4.ipynb
'QCNN Simulations.ipynb'     Untitled18.ipynb   Untitled5.ipynb
 QuantumCNN.ipynb	     Untitled19.ipynb   Untitled6.ipynb
'Quantum NN.ipynb'	     Untitled1.ipynb    Untitled7.ipynb
 Untitled		     Untitled20.ipynb   Untitled8.ipynb
 Untitled0.ipynb	     Untitled21.ipynb   Untitled9.ipynb
 Untitled10.ipynb	     Untitled22.ipynb
 Untitled11.ipynb	     Untitled23.ipynb


In [4]:
!pip install qiskit

In [5]:
!pip install qiskit-terra

In [15]:
import matplotlib.pyplot as plt
import numpy as np

from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name

seed = 10599
aqua_globals.random_seed = seed


%matplotlib inline

import numpy as np
import seaborn as sns
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
import time

In [7]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/FTP-Patator.csv')
df1 = df.iloc[1:500]
df2 = df.iloc[4221:4750]
df = pd.concat([df1, df2], ignore_index=True)



In [8]:
feature_list={"Fwd Packet Length Max","Fwd Packet Length Std","Fwd Packet Length Mean","Bwd Packet Length Std","Label"}
df=df.fillna(0)
attack_or_not=[]
for i in df["Label"]: #makine öğrenme algoritmasında kullanılmak üzere normal etiketi "1" ve saldırı etiketini "0" olarak değiştirir
    if i =="BENIGN":
        attack_or_not.append(1)
    else:
        attack_or_not.append(0) 
df["Label"]=attack_or_not
y = df["Label"]
del df["Label"]
feature_list.remove('Label')
X = df[feature_list]

# cross-validation
X_train, X_test, y_train, y_test = train_test_split(X, y,#  data (X) and labels (y) are divided into 2 parts to be sent to the machine learning algorithm (80% train,%20 test). 
    test_size = 0.20)#  So, in total there are 4 tracks: training data(X_train), training tag (y_train), test data(X_test) and test tag(y_test).


In [9]:
#Quantum için Önişleme
std_scaler = StandardScaler().fit(X_train)
X_train = std_scaler.transform(X_train)
X_test = std_scaler.transform(X_test)

In [10]:
pca = PCA(n_components=2).fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [11]:
samples = np.append(X_train, X_test, axis=0)
minmax_scaler = MinMaxScaler((-1, 1)).fit(samples)
X_train = minmax_scaler.transform(X_train)
X_test = minmax_scaler.transform(X_test)

In [12]:
labels = ['0', '1']
training_input = {key: (X_train[y_train == k, :]) for k, key in enumerate(labels)}
test_input = {key: (X_test[y_test == k, :]) for k, key in enumerate(labels)}


print(len(training_input))
print(len(test_input))

2
2


In [16]:
seconds=time.time()#time stamp for all processing time

In [17]:
datapoints, class_to_label = split_dataset_to_data_and_labels(test_input)
feature_dim = 2 
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2, entanglement='linear')
print(feature_map)

     ┌───┐┌──────────────┐                                           ┌───┐»
q_0: ┤ H ├┤ U1(2.0*x[0]) ├──■─────────────────────────────────────■──┤ H ├»
     ├───┤├──────────────┤┌─┴─┐┌───────────────────────────────┐┌─┴─┐├───┤»
q_1: ┤ H ├┤ U1(2.0*x[1]) ├┤ X ├┤ U1(2.0*(π - x[0])*(π - x[1])) ├┤ X ├┤ H ├»
     └───┘└──────────────┘└───┘└───────────────────────────────┘└───┘└───┘»
«     ┌──────────────┐                                           
«q_0: ┤ U1(2.0*x[0]) ├──■─────────────────────────────────────■──
«     ├──────────────┤┌─┴─┐┌───────────────────────────────┐┌─┴─┐
«q_1: ┤ U1(2.0*x[1]) ├┤ X ├┤ U1(2.0*(π - x[0])*(π - x[1])) ├┤ X ├
«     └──────────────┘└───┘└───────────────────────────────┘└───┘


In [27]:
qsvm = QSVM(feature_map, training_input, test_input, datapoints[0])
backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

result = qsvm.run(quantum_instance)

qasm_simulator


In [19]:
print("mission accomplished!")
print("Total operation time: = ",time.time()- seconds ,"seconds")

mission accomplished!
Total operation time: =  3933.625867843628 seconds


In [20]:
print("accuracy: {}".format(result['testing_accuracy']))

accuracy: 0.6941747572815534


In [21]:
from qiskit.aqua.algorithms import SklearnSVM

result = SklearnSVM(training_input, test_input, datapoints[0]).run()

In [22]:
print("mission accomplished!")
print("Total operation time: = ",time.time()- seconds ,"seconds")

mission accomplished!
Total operation time: =  3950.896164894104 seconds


In [23]:
print("accuracy: {}".format(result['testing_accuracy']))

accuracy: 0.6601941747572816
